In [1]:
import os
from datetime import datetime
from dateutil.parser import parse
import time

import numpy as np
import pandas as pd
import scipy.stats as scs
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pprint import pprint

from src.nft_analytics import NFT_Analytics
from src.ethereum_api import EthereumAPI

In [2]:
gaa = NFT_Analytics("0xA0F38233688bB578c0a88102A95b846c18bc0bA7")
eth_api = EthereumAPI()
DATA_FOLDER = os.path.join("data", "GreatApeSociety")
RESULT_FOLDER = os.path.join("results", "GreatApeSociety")

2021-08-17 03:30:27,208 - src.ethereum_api - INFO - Connected to Infura endpoint.


# Fetch all GreatApe data from OpenSea

Note that this can fail if OpenSea decides to rate limit. Works best at night when the load on their servers is low.

In [ ]:
asset_data = gaa.fetch_data(max_offset=10000)

In [ ]:
len(asset_data)

# Save list to json file for further use

In [ ]:
gaa.save_json(asset_data, filename=os.path.join(DATA_FOLDER, "data.json")

# Load json file

In [ ]:
asset_data = gaa.load_json(filename=os.path.join(DATA_FOLDER, "data.json"))

## Scatter plot of listing prices with time

In [ ]:
created_dates = []
listing_prices = []
for asset in asset_data:
    if asset["sell_orders"]:
        created_dates.append(parse(asset["sell_orders"][0]["created_date"]))
        listing_prices.append(float(asset["sell_orders"][0]["base_price"]) / 1e18)
listing_prices = np.array(listing_prices)

In [ ]:
fig, ax = plt.subplots()

dates = matplotlib.dates.date2num(created_dates)
ax.plot_date(dates, listing_prices, 'o', alpha=0.5)
fig.autofmt_xdate()
ax.set_ylim(-0.01, 0.1)
ax.set_ylabel("Listing Price (ETH)")
ax.axhline(0.06, color="tab:orange", linestyle="--", label="Current OpenSea floor price")
ax.set_title("Variation of listing price with time")
ax.legend()

fig.savefig(os.path.join(RESULT_FOLDER, "listing_price_variation_with_time_near_floor.jpg"), bbox_inches="tight", dpi=600)

In [ ]:
# Cost to cleanup floor
np.sum(listing_prices[listing_prices < 0.06])

## Histogram of listing prices

In [ ]:
upper_limit = 5

fig, ax = plt.subplots()
_, bins, _ = ax.hist(listing_prices, histtype='stepfilled', density=True, bins=60, range=[0, upper_limit], alpha=0.5)

params = scs.lognorm.fit(listing_prices[listing_prices < upper_limit])
ax.plot(bins, scs.lognorm.pdf(bins, *params), 'tab:red', linewidth=2, label=f"Lognormal distribution\nMean = {params[2]:.2f} ETH")

ax.set_xlabel("Listing Price (ETH)")
ax.set_ylabel("No. of listings (norm.)")
ax.legend()

fig.savefig(os.path.join(RESULT_FOLDER, "histogram_listing_price_variation_near_floor.jpg"), bbox_inches="tight", dpi=600)

# Sale price with time

In [ ]:
sale_prices = []
timestamps = []

for asset in asset_data:
    if asset["last_sale"]:
        if asset["last_sale"]["event_type"] == "successful":
            timestamps.append(parse(asset["last_sale"]["transaction"]["timestamp"]))
            sale_prices.append(float(asset["last_sale"]["total_price"]) / 1e18)
            
sale_prices = np.array(sale_prices)

In [ ]:
fig, ax = plt.subplots()

dates = matplotlib.dates.date2num(timestamps)
ax.plot_date(dates, sale_prices, 'o', alpha=0.5)
fig.autofmt_xdate()
ax.set_ylim(-0.01, 0.1)
ax.set_ylabel("Sale Price (ETH)")
ax.axhline(0.06, color="tab:orange", linestyle="--", label="Current OpenSea floor price")
ax.set_title("Variation of sale price with time")
ax.legend()

fig.savefig(os.path.join("results", "sale_price_variation_with_time_near_floor.jpg"), bbox_inches="tight", dpi=600)

# Histogram of sale prices

In [ ]:
upper_limit = 1

fig, ax = plt.subplots()
_, bins, _ = ax.hist(sale_prices, density=True, bins=60, range=[0, upper_limit], alpha=0.5)

params = scs.lognorm.fit(sale_prices[sale_prices < upper_limit])
ax.plot(bins, scs.lognorm.pdf(bins, *params), 'tab:red', linewidth=2, label=f"Lognormal distribution\nMean = {params[2]:.2f} ETH")

ax.set_xlabel("Sale Price (ETH)")
ax.set_ylabel("No. of sales (norm.)")
ax.legend()

fig.savefig(os.path.join(RESULT_FOLDER, "histogram_sale_price_variation_near_floor.jpg"), bbox_inches="tight", dpi=600)

# Time histogram of number of sales (defined by timestamps within bin)

In [ ]:
upper_limit = 1

fig, ax = plt.subplots()

unix_timestamps = []

for times in timestamps:
    unix_timestamps.append(times.timestamp())

mpl_data = mdates.epoch2num(unix_timestamps)
_, bins, _ = ax.hist(mpl_data, bins=50, alpha=0.5)

ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m'))
ax.set_ylabel("No. of sales")

fig.savefig(os.path.join(RESULT_FOLDER, "histogram_number_of_sales_with_time.jpg"), bbox_inches="tight", dpi=600)

# Time histogram of number of sales over a day (defined by timestamps within bin)

In [ ]:
fig, ax = plt.subplots()

hours = []

for times in timestamps:
    hours.append(times.hour)
    
_, bins, _ = ax.hist(hours, bins=24, alpha=0.5)

ax.set_ylabel("No. of sales")
ax.set_xlabel("Time of day in hours (UTC)")

fig.savefig(os.path.join(RESULT_FOLDER, "histogram_number_of_sales_with_time_of_day.jpg"), bbox_inches="tight", dpi=600)

# Median price of traits

In [ ]:
traits = {
    "Clothes" : "Hufflepuff",
    "Eyes": "Orange_Glasses",
    "Fur": "Gold_Skin",
    "Background": "Carolina",
    "Mouth": "Laughing_Mouth",
    "Earring": "Green_Red_Earring",
    "Hat": "Fire_Hair"
}

prices = gaa.get_median_prices(asset_data, traits)
print(f"Average price = {np.average(prices):.2f}, Max price = {np.max(prices):.2f}, Min price = {np.min(prices):.2f}")

# Most valuable traits based on listing price

In [ ]:
for trait in traits.keys():
    trait_dict = gaa.get_trait_type_median_price(asset_data, trait)
    val = {k: trait_dict[k] for k in list(trait_dict)[:1]}
    print(f"{trait}: {val}")

# ETH in seller's wallet

In [ ]:
eth_in_wallet_buyer = []
eth_in_wallet_seller = []

for asset in asset_data:
    if asset["last_sale"]:
        if asset["last_sale"]["event_type"] == "successful":
            seller_address = asset["last_sale"]["transaction"]["from_account"]["address"]
            eth_in_wallet_seller.append(eth_api.get_eth_balance(seller_address))
            
            # buyer_address = asset["last_sale"]["transaction"]["to_account"]["address"]
            # eth_in_wallet_buyer.append(get_total_eth_weth_balance(weth_contract, buyer_address))

In [ ]:
gaa.save_json(eth_in_wallet_seller, filename=os.path.join(DATA_FOLDER, "eth_in_wallet_seller.json"))

In [ ]:
eth_in_wallet_seller = gaa.load_json(filename=os.path.join(DATA_FOLDER, "eth_in_wallet_seller.json"))
eth_in_wallet_seller = np.array(eth_in_wallet_seller)

In [ ]:
fig, ax = plt.subplots()

upper_limit = 20

_, bins, _ = ax.hist(eth_in_wallet_seller, density=True, bins=100, alpha=0.5, range=[0, upper_limit], label=f"Median = {np.median(eth_in_wallet_seller):.2f} ETH")

print(f"Under {upper_limit} ETH = {len(eth_in_wallet_seller[eth_in_wallet_seller < upper_limit]) / len(eth_in_wallet_seller) * 100:.2f}%")
print(f"Over {upper_limit} ETH = {len(eth_in_wallet_seller[eth_in_wallet_seller > upper_limit]) / len(eth_in_wallet_seller) * 100:.2f}%")

ax.set_ylabel("No. of wallets")
ax.set_xlabel("ETH in wallet")
ax.legend()

fig.savefig(os.path.join(RESULT_FOLDER, "eth_in_seller_wallet.jpg"), bbox_inches="tight", dpi=600)

# Average ETH/WETH for GreatApe owner

In [ ]:
addresses = []

for asset in asset_data:
    addresses.append(asset["owner"]["address"])

In [ ]:
for idx in range(0, len(addresses), len(addresses)//100):
    print(addresses[idx], )

In [ ]:
weth_contract = eth_api.get_contract("0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2")

In [ ]:
eth_in_address = []

for idx in range(0, len(addresses), len(addresses)//100):
    eth_in_address.append(eth_api.get_total_eth_weth_balance(weth_contract, addresses[idx]))
eth_in_address = np.array(eth_in_address)

In [ ]:
gaa.save_json(list(eth_in_address), filename=os.path.join(DATA_FOLDER, "eth_in_address.json"))

In [ ]:
eth_in_address = gaa.load_json(filename=os.path.join(DATA_FOLDER, "eth_in_address.json"))

In [ ]:
for idx1, idx2 in zip(range(0, len(addresses), len(addresses)//100), range(len(eth_in_address))):
    if eth_in_address[idx2] > 15:
        print(addresses[idx1], eth_in_address[idx2])

0x4985496569c9a5ccf8b612cb40ba8f4b94a44534 (45 ETH, >6 Apes)
0x58d0f3da9c97de3c39f481e146f3568081d328a2 (53 ETH, >10 Apes)
0xd7f5663e7e93f0c1fb9347c9aef4a17f23522a37 (35 ETH, >70 Apes)
0x6ef9dca82362509cd878051d1fdc6db12dda2989 (19 ETH, >25 Apes)
0xbdbfe5f4d8775472b0398f88fc378ec2e5518558 (19 ETH, >20 Apes)
0x704c8c855765b4c053cc2dc02b3f318caf2ab732 (15 ETH, >16 Apes)
0xa13bf3555194c8a65c2ac6324731852d6ecf2638 (17 ETH, >12 Apes)

In [ ]:
fig, ax = plt.subplots()

upper_limit = 55

_, bins, _ = ax.hist(eth_in_address, bins=20, alpha=0.5, range=[0, upper_limit], label=f"Median = {np.median(eth_in_address):.2f} ETH")

ax.set_ylabel("No. of wallets")
ax.set_xlabel("ETH/WETH in wallet")
#ax.set_yscale("log")
ax.legend()
#fig.savefig(os.path.join(RESULT_FOLDER, "eth_weth_holding_of_great_ape_owners.jpg"), bbox_inches="tight", dpi=600)

In [ ]:
print(np.median(eth_in_address), np.mean(eth_in_address), np.max(eth_in_address))

In [ ]:
upper_limit = 0.4
print(f"Under {upper_limit} ETH = {len(eth_in_address[eth_in_address < upper_limit]) / len(eth_in_address) * 100}%")
print(f"Over {upper_limit} ETH = {len(eth_in_address[eth_in_address > upper_limit]) / len(eth_in_address) * 100}%")

In [ ]:
events_data = gaa.fetch_events(max_offset=10000)

In [ ]:
len(events_data)

In [ ]:
gaa.save_json(events_data, filename=os.path.join(DATA_FOLDER, "events.json"))

In [ ]:
events_data = gaa.load_json(filename=os.path.join(DATA_FOLDER, "events.json"))

In [ ]:
for event in events_data:
    pprint(event["created_date"])
    break

In [ ]:
bids = []
created_dates = []

for event in events_data:
    if event["bid_amount"]:
        created_dates.append(parse(event["created_date"]))
        bids.append(float(event["bid_amount"]) / 1e18)
bids = np.array(bids)

In [ ]:
fig, ax = plt.subplots()
#ax.set_yscale("log")
dates = matplotlib.dates.date2num(created_dates)
ax.plot_date(dates, bids, 'o', alpha=0.5)
fig.autofmt_xdate()
ax.set_ylabel("Bid Price (ETH)")
ax.axhline(0.06, color="tab:orange", linestyle="--", label="Current OpenSea floor price")
ax.legend()
ax.set_title(f"Variation of bids with time (Bids under floor = {len(bids[bids < 0.06]) / len(bids) * 100:.1f}%)")
fig.savefig(os.path.join(RESULT_FOLDER, "bid_price_variation_with_time.jpg"), bbox_inches="tight", dpi=600)

# Great Apes held by addresses

In [ ]:
def remove_duplicates_from_list(x: list):
    return list(dict.fromkeys(x))

In [ ]:
ape_contract = eth_api.get_contract("0xA0F38233688bB578c0a88102A95b846c18bc0bA7")

In [ ]:
df = pd.DataFrame(columns=["Address", "ETH_balance", "Great_apes"])

addresses = remove_duplicates_from_list(addresses)

for idx, address in enumerate(addresses):
    df.loc[idx] = [address, eth_api.get_eth_balance(address), eth_api.get_contract_balance(ape_contract, address)]

In [ ]:
df.sort_values("Great_apes", ascending=False)

In [ ]:
etherscan_links = []
for address in df["Address"]:
    etherscan_links.append(f"https://etherscan.io/address/{address}")
df["Etherscan_link"] = etherscan_links

opensea_links = []
for address in df["Address"]:
    opensea_links.append(f"https://opensea.io/{address}")
df["OpenSea_link"] = opensea_links

In [ ]:
df.to_excel(os.path.join(RESULT_FOLDER, "ape_holders_addresses_balances_and_holdings.xlsx"), index=False)

In [ ]:
df = pd.read_excel(os.path.join(RESULT_FOLDER, "ape_holders_addresses_balances_and_holdings.xlsx"))

In [ ]:
fig, ax = plt.subplots()

_, bins, _ = ax.hist(df["Great_apes"], bins=30, alpha=0.5, range=[0, 30], label=f"Median = {int(np.median(df['Great_apes']))} Apes")

ax.set_ylabel("No. of wallets")
ax.set_xlabel("Apes in wallet")
ax.legend()
#fig.savefig(os.path.join(RESULT_FOLDER, "eth_weth_holding_of_great_ape_owners.jpg"), bbox_inches="tight", dpi=600)

In [ ]:
df.sort_values("ETH_balance", ascending=False).head(10)

In [ ]:
df.sort_values("Great_apes", ascending=False).head(10)

# Further analysis